In [1]:
# Nuskaito ir parodo anotacijas visiems failams iš records_npy.

import pandas as pd
import numpy as np

import json, sys
from pathlib import Path
from icecream import ic
from collections import Counter

from zive_util_vu import get_rec_userNr, get_rec_Nr, create_SubjCode, read_rec_attrib
from zive_util_vu import get_annotations_distribution

import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ", my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_2022_RUDUO_2'

# Aplankas su  EKG įrašais ir anotacijomis (.json)
# rec_folder = 'records'
rec_folder = 'records_npy'

# ///////////////////////////////////////////////////////////////////////////////////////////
print('\nSkriptas parodo anotacijų statistiką vietinėje ZIVE talpykloje')

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns", 20)
pd.set_option('display.width', 1000)
# How to print an entire pandas DataFrame in Python
# https://www.kite.com/python/answers/how-to-print-an-entire-pandas-dataframe-in-python

# Nuoroda į aplanką su vietiniu EKG duomenų rinkiniu
rec_dir = Path(Duomenu_aplankas, db_folder, rec_folder) # vietinė talpykla su jau esamais duomenimis arba tuščia
print( 'Vietinės talpyklos aplankas:', db_folder)

# Nuskaitomas įrašų sąrašas, suformuojamas atitinkamas dataframe df_list
file_path = Path(rec_dir, 'list_npy.json')
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())
df_list = pd.json_normalize(data, record_path =['data'])
print(df_list.head())


OS in my system :  win32

Skriptas parodo anotacijų statistiką vietinėje ZIVE talpykloje
Vietinės talpyklos aplankas: DUOM_2022_RUDUO_2
   SubjCode    file_name                    userId               recordingId     N  S  V  U  incl  flag               recorded_at
0     10000  1630673.825  6034c808d6c2740008035ede  6133101f3d08d40385cdc8ce   762  4  0  0     0     0  2021-09-03T12:57:05.000Z
1     10001  1626941.468  6034c808d6c2740008035ede  60f945e187cf66e97e37810e   859  2  0  0     0     0  2021-07-22T08:11:08.000Z
2     10002  1632923.661  6034c808d6c2740008035ede  615516b62312e50c3f168e38  4849  1  0  0     0     0  2021-09-29T13:54:21.000Z
3     10010  1621694.321  60a917b354352a3df86dc1f2  60a92aa454352a03116dc1f5  6009  0  9  0     0     0  2021-05-22T14:38:41.000Z
4     10011  1626330.744  60a917b354352a3df86dc1f2  60efffba87cf660aac377fe7   788  5  0  4     0     0  2021-07-15T06:32:24.000Z


In [2]:
# # Surandame įrašų, kuriuose yra nurodytos anotacijos, skaičius.
#  Tuo pačiu surandame, ar nėra nereikalingų anotacijų.

all_lst = []

for index, row in df_list.iterrows():
    SubjCode =  row["SubjCode"]
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)
    smb_lst = list(Counter(atr_symbol))
    # print (index, SubjCode, smb_lst)
    for i in range(len(smb_lst)):
        all_lst.append(smb_lst[i])

# print(all_lst)
smb_lst = Counter(all_lst)
print("\nĮrašų, kuriuose yra nurodytos anotacijos, skaičius:")
print(smb_lst)



Įrašų, kuriuose yra nurodytos anotacijos, skaičius:
Counter({'N': 1086, 'V': 743, 'S': 378, 'U': 20})


In [4]:
# Pagrindinė

# Užduodame anotacijų sąrašą ir surandame anotacijų pasiskirstymą per įrašus
annot_list = ['N', 'S', 'V', 'U']

# Suformuojame sąraše naują kolonėlę su ekstrasistolių kiekiu
df_list['E'] = df_list['S'] + df_list['V']

filename = 'SurusiuotiIrasai'
if (OS == 'Windows'): 
    # Windows - įrašome Excel lentelę su rezultatais
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html
    
    filepath = filename + '.xlsx'
    with pd.ExcelWriter(filepath) as writer:
        df_list.to_excel(writer, sheet_name='sarasas')
    print(f'\nAtrinktų įrašų sąrašas įrašytas į:  {filepath}')

# Apskaičiuojame ir išvedame statistiką
df_sum = df_list.groupby('userId').sum()

# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
counts = df_list['userId'].value_counts()
print(f'\nViso pacientų: {len(counts)}  EKG įrašų: {len(df_list)}')

# print(f'\nAtrinktų įrašų skaičiaus pasiskirstymas per pacientus (surūšiuotas)')
# print(f'userId                      Įrašų skaičius')
# print(counts)

counts = counts.rename("counts")
frames = [df_sum, counts]
result = pd.concat(frames, axis = 1)
result.index.rename ('userID', inplace= True)
result.drop(labels=['incl'], axis=1, inplace=True)
result = result.sort_values(by='counts', ascending=False)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/

print("\nAtrinktų pacientų sąrašas (surušiuotas pagal paciento įrašų skaičių - counts)")
print(result)

# Susumuojamos anotacijos
suma = result.iloc[:,0:7].sum()
print('\n',' '*18, 'sum: ',str(suma.tolist())[1:-1])

selected_columns = result[['E']]
result_new = selected_columns.copy()
# print(result_new)

sorted_group = result_new.sort_values(by='E', ascending=False)
print("\nAtrinktų ir surūšiuotų pagal bendrą ekstrasistolių kiekį E pacientų sąrašas")
print(sorted_group)


# Išplėstinė statistika
print("\nIšplėstinis pacientų sąrašas, surūšiuotas pagal įrašų skaičių ir ")
print("šių pacientų EKG įrašų sąrašai, surušiuoti pagal bendrą ekstrasistolių kiekį E")
grouped  = df_list.groupby('userId')
lst_grouped  = df_list.groupby('userId').size().sort_values(ascending=False)
userIds = lst_grouped.index

for userId in userIds:
    group = grouped.get_group(userId)
    # print(type(userId), userId)
    userNr = get_rec_userNr(rec_dir, userId)
    print(f"\nuserId: {userId}  userNr: {userNr} EKG įrašų: {len(group)} " )
    tit1 = f"{'recordingNr':>0} {'recordingId':>16} {'file_name':>27} {'N':>10}" 
    tit2 =f" {'S':>7}   {'V':>6}   {'U':>6}  {'E':>6}" 
    print(tit1+tit2)
    for idx, row in group.iterrows():
        recordingId = row['recordingId']
        userNr, recordingNr = get_rec_Nr(rec_dir, userId, recordingId)
        SubjCode = create_SubjCode(userNr, recordingNr)
        str1 =f"{SubjCode:>11} {recordingId:>29} {str(row['file_name']):>16} {int(row['N']):>8} " 
        str2 =f" {int(row['S']):>6}   {int(row['V']):>6}   {int(row['U']):>6}  {int(row['E']):>6}" 
        print(str1+str2)




Atrinktų įrašų sąrašas įrašytas į:  SurusiuotiIrasai.xlsx

Viso pacientų: 102  EKG įrašų: 1087

Atrinktų pacientų sąrašas (surušiuotas pagal paciento įrašų skaičių - counts)
                          SubjCode      N     S     V  U  flag     E  counts
userID                                                                      
61632bd41e32554ace017bc0  47722816  89146     1   718  0     0   719     137
6144c682bd0cc5acb7275368  45966080  87673     4  4460  2     0  4464     136
6144c588bd0cc52ba0275362   5686080  38541     0  1156  0     0  1156      65
61632afa1e325516b4017bbb   4950396  47016    29   254  1     0   283      57
6144c4fbbd0cc552e427535f   3242420  28108     2  3760  0     0  3762      41
61632ace1e32557a62017bba   2575561  33778  1864     9  1     0  1873      34
613b1d673d08d4d1f3cdc8f8   2018406  21721   126     0  5     0   126      29
617509a66d4fb46d3b637744   1980178  21182     4    95  0     0    99      28
6190d4b23cd1d29db2303ce9   1685100  14077   204     4  